In [150]:
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from timedelta_formatter import strfdelta
import numpy as np
import pandas as pd
from create_connection import get_data
import matplotlib.pyplot as plt
import seaborn as sns
from create_connection import get_data
from userlist import blokparti_user_list
%matplotlib inline
from IPython.display import display
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
sns.set_style('darkgrid')

In [151]:
get_data()
gsheet_url = "https://docs.google.com/spreadsheets/d/1qxoNvkSvDfmIhoJgs9XFucN01kCZsd-1TkVJSzRj-CA/edit#gid=2051699675"

# import data
accounts = pd.read_csv("csv_files/accounts.csv", encoding="utf-8", dtype={"phone": str})
parties = pd.read_csv(
    "csv_files/parties.csv",
    encoding="utf-8",
    parse_dates=["begin_time", "end_time", "start_time", "updated_at"],
)
party_user = pd.read_csv(
    "csv_files/party_user.csv",
    encoding="utf-8",
    parse_dates=["created_at", "updated_at"],
)
users = pd.read_csv("csv_files/users.csv", encoding="utf-8")
user_sessions = pd.read_csv("csv_files/user_sessions.csv", encoding="utf-8")
party_played_items = pd.read_csv(
    "csv_files/party_played_items.csv",
    encoding="utf-8",
    parse_dates=["played_at", "updated_at"],
)
messages = pd.read_csv(
    "csv_files/messages.csv", encoding="utf-8", parse_dates=["created_at"]
)

# count number of parties host and duration
dff = pd.merge(
    party_user[["party_id", "user_id", "created_at", "role", "updated_at"]],
    parties[["id", "begin_time", "end_time", "updated_at", "creator_id"]],
    how="left",
    left_on="party_id",
    right_on="id",
)
del dff["id"]

dff.columns = [
    "party_id",
    "user_id",
    "user_party_created_at",
    "role",
    "user_party_updated_at",
    "party_begin_time",
    "party_end_time",
    "party_updated_at",
    "creator_id",
]
dff_ = pd.merge(
    dff,
    users[["id", "account_id", "username"]],
    how="left",
    left_on="user_id",
    right_on="id",
)
del dff_["id"]
dff_ = pd.merge(
    dff_,
    users[["id", "account_id", "username"]],
    how="left",
    left_on="creator_id",
    right_on="id",
    suffixes=("_pu", "_creator"),
)
del dff_["id"]


def party_duration(row):
    if row["role"] == "guest":
        if row["user_party_updated_at"] is pd.NaT and row["party_end_time"] is pd.NaT:
            return row["party_updated_at"] - row["user_party_created_at"]
        elif (
            row["user_party_updated_at"] is pd.NaT
            and row["party_end_time"] is not pd.NaT
        ):
            return row["party_end_time"] - row["user_party_created_at"]
        elif (
            row["user_party_updated_at"] is not pd.NaT
            and row["party_end_time"] is not pd.NaT
        ):
            return row["user_party_updated_at"] - row["user_party_created_at"]
        else:
            return pd.NaT
    elif row["role"] == "host":
        if row["party_end_time"] is pd.NaT:
            return row["party_updated_at"] - row["party_begin_time"]
        else:
            return row["party_end_time"] - row["party_begin_time"]


dff_["party_duration"] = dff_.apply(party_duration, axis=1)
dff_ = dff_.dropna(subset=["party_duration"])
dff_["start_date"] = dff_["party_begin_time"].dt.date
dff_ = pd.merge(
    dff_,
    accounts[["id", "phone"]],
    how="left",
    left_on="account_id_pu",
    right_on="id",
)
del dff_["id"]

df = dff_[~(dff_["phone"].isin(blokparti_user_list)) & (dff_["party_begin_time"] > "2021-11-29")].copy()
df["week"] = df["party_begin_time"].dt.isocalendar().week
df["year"] = df["party_begin_time"].dt.isocalendar().year

# starts pivoting
basic_dff = pd.pivot_table(
    df,
    index=["start_date", "week", "year", "username_pu", "phone"],
    values=["party_id", "party_duration"],
    aggfunc={"party_duration": np.sum, "party_id": len},
    columns="role",
)
basic_dff = basic_dff.reset_index()
basic_dff.columns = ["_".join(a) for a in basic_dff.columns.to_flat_index()]
basic_dff.party_duration_guest = basic_dff.party_duration_guest.fillna(
    np.timedelta64(0)
)
basic_dff.party_duration_host = basic_dff.party_duration_host.fillna(np.timedelta64(0))
basic_dff["all_duration"] = (
    basic_dff.party_duration_guest + basic_dff.party_duration_host
)


def time_range(timed):
    if timed < np.timedelta64(30, "m"):
        return "< 30m"
    elif timed < np.timedelta64(60, "m"):
        return "30m-60m"
    else:
        return ">1h"


basic_dff["duration_range"] = basic_dff["all_duration"].apply(time_range)

fmt = "{D} days {H}h{M}m{S}s"
basic_dff["party_duration_guest_text"] = basic_dff.party_duration_guest.apply(
    lambda x: strfdelta(x, fmt)
)
basic_dff["party_duration_host_text"] = basic_dff.party_duration_host.apply(
    lambda x: strfdelta(x, fmt)
)
basic_dff["all_duration_text"] = basic_dff.all_duration.apply(lambda x: strfdelta(x, fmt))

#turn timedelta into seconds
basic_dff["party_duration_guest"] = basic_dff.party_duration_guest.dt.total_seconds()
basic_dff["party_duration_host"] = basic_dff.party_duration_host.dt.total_seconds()
basic_dff["all_duration"] = basic_dff.all_duration.dt.total_seconds()

basic_dff = basic_dff.sort_values(["start_date_", "username_pu_"], ascending=False)
basic_dff = basic_dff.replace("0 days 0h0m0s", "")
gc = gspread.service_account()
sh = gc.open_by_url(gsheet_url)
worksheet = sh.worksheet("Detail")

intern_names = get_as_dataframe(
    sh.worksheet("Intern_name"),
    dtype={"phone_": str},
    usecols=[0, 1, 2, 3],
    skip_blank_line=True,
    evaluate_formulas=True,
)
basic_dff_ = pd.merge(
    basic_dff,
    intern_names,
    how="left",
    left_on=["username_pu_", "phone_"],
    right_on=["username", "phone_"],
)
basic_dff_ = basic_dff_[
    [
        "week_",
        "year_",
        "start_date_",
        "username_pu_",
        "phone_",
        "name",
        "party_id_host",
        "party_id_guest",
        "party_duration_host_text",
        "party_duration_guest_text",
        "all_duration_text",
        "party_duration_host",
        "party_duration_guest",
        "all_duration",
        "duration_range",
    ]
]
set_with_dataframe(worksheet, basic_dff_, row=2, col=2, include_column_header=False)

#get party hosts
dfd = df.sort_values(["start_date", "username_pu", "username_creator"], ascending=False)[["start_date","username_pu", "role", "username_creator"]]
dfd = dfd[dfd.role=="guest"].drop_duplicates()
dfd['party_host'] = dfd.groupby(['start_date', "username_pu"])['username_creator'].transform(lambda x : ', '.join(x))
dfdf = pd.merge(basic_dff_, dfd, how="left", left_on=["username_pu_", "start_date_"], right_on=["username_pu", "start_date"])
dfdf = dfdf[["start_date_", "username_pu_", "party_host"]].drop_duplicates()
set_with_dataframe(
    worksheet, dfdf[["party_host"]], row=2, col=20, include_column_header=False
)

# participants
participants = pd.pivot_table(
    df,
    index=["start_date", "username_creator"],
    values=["user_id"],
    aggfunc=len,
    columns="role",
).reset_index()

participants.columns = ["_".join(a) for a in participants.columns.to_flat_index()]
participants = participants.sort_values(
    ["start_date_", "username_creator_"], ascending=False
)
dfg = pd.merge(
    basic_dff,
    participants,
    how="left",
    left_on=["username_pu_", "start_date_"],
    right_on=["username_creator_", "start_date_"],
)
set_with_dataframe(
    worksheet, dfg[["user_id_guest"]], row=2, col=17, include_column_header=False
)

# message
message = pd.merge(
    parties[["id", "start_time"]],
    messages[["conversation_id", "id", "content", "user_id", "created_at"]],
    how="right",
    left_on="id",
    right_on="conversation_id",
)
message.columns = [
    "party_id",
    "party_start_time",
    "conversation_id",
    "message_id",
    "content",
    "user_id",
    "message_created_at",
]
message = pd.merge(
    message,
    users[["id", "account_id", "username"]],
    how="left",
    left_on="user_id",
    right_on="id",
)
message_ = pd.merge(
    message,
    accounts[["id", "email", "phone"]],
    how="left",
    left_on="account_id",
    right_on="id",
)
del message_["id_x"]
del message_["id_y"]
del message_["email"]
message_["message_date"] = message_["message_created_at"].dt.date
message_ = message_[~message_.phone.isin(blokparti_user_list)]

message_df = pd.pivot_table(
    message_, index=["message_date", "username"], values="message_id", aggfunc=len
)
message_df = message_df.reset_index().sort_values(
    ["message_date", "username"], ascending=False
)
message_df = pd.merge(
    basic_dff,
    message_df[["message_date", "username", "message_id"]],
    how="left",
    left_on=["start_date_", "username_pu_"],
    right_on=["message_date", "username"],
)
set_with_dataframe(
    worksheet, message_df[["message_id"]], row=2, col=18, include_column_header=False
)

# playlist_items
playlist_df = pd.merge(
    df,
    party_played_items[["party_id", "played_at", "id", "updated_at"]],
    how="left",
    left_on="party_id",
    right_on="party_id",
)

playlist_pivot = (
    pd.pivot_table(
        playlist_df, index=["start_date", "username_pu"], values="id", aggfunc=len
    )
    .reset_index()
    .sort_values(["start_date", "username_pu"], ascending=False)
)
playlist_gf = pd.merge(
    basic_dff,
    playlist_pivot,
    how="left",
    left_on=["start_date_", "username_pu_"],
    right_on=["start_date", "username_pu"],
)
set_with_dataframe(
    worksheet, playlist_gf[["id"]], row=2, col=19, include_column_header=False
)



Processing time to get data is 65.90274596214294s


In [152]:
from pathlib import Path
from flatten_nested_json_columns import flatten_nested_json_df
from get_id_frontend import get_global_id
folder = 'json_files'
files = [f for f in Path(folder).glob('*') if f.is_file()]


In [171]:
glued_df = pd.DataFrame()
for file in files:
    x = pd.read_json(file, convert_dates = ["event_timestamp", "event_date"])
    glued_df = pd.concat([glued_df,x],axis=0)

In [172]:
glued_df.head()

,event_timestamp,event_name,event_params,operating_system
0,2021-12-18 01:18:10.777001,app_update,"[{'key': 'previous_app_version', 'value': {'st...",iOS
1,2021-12-18 01:18:10.059000,session_start,"[{'key': 'ga_session_id', 'value': {'string_va...",iOS
2,2021-12-18 02:48:18.352000,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",iOS
3,2021-12-18 08:18:06.969000,session_start,"[{'key': 'firebase_event_origin', 'value': {'s...",iOS
4,2021-12-18 09:48:39.282000,session_start,"[{'key': 'firebase_event_origin', 'value': {'s...",iOS


In [173]:

glued_df = flatten_nested_json_df(glued_df)

original shape: (6913, 5)
original columns: Index(['index', 'event_timestamp', 'event_name', 'event_params', 'operating_system'], dtype='object')
lists: ['event_params'], dicts: []
exploding: event_params
lists: [], dicts: ['event_params']
flattening: event_params
lists: [], dicts: []
final shape: (50805, 9)
final columns: Index(['index', 'event_timestamp', 'event_name', 'operating_system', 'event_params.key', 'event_params.value.string_value', 'event_params.value.int_value', 'event_params.value.float_value', 'event_params.value.double_value'], dtype='object')


In [174]:
glued_df.head()

,index,event_timestamp,event_name,operating_system,event_params.key,event_params.value.string_value,event_params.value.int_value,event_params.value.float_value,event_params.value.double_value
0,0,2021-12-18 01:18:10.777001,app_update,iOS,previous_app_version,2.0.0,None,None,None
0,0,2021-12-18 01:18:10.777001,app_update,iOS,ga_session_id,None,1639790290,None,None
0,0,2021-12-18 01:18:10.777001,app_update,iOS,firebase_event_origin,auto,None,None,None
0,0,2021-12-18 01:18:10.777001,app_update,iOS,ga_session_number,None,114,None,None
1,1,2021-12-18 01:18:10.059000,session_start,iOS,ga_session_id,None,1639790290,None,None


In [154]:
glued_df[
    (glued_df["event_params.key"] == "label") 
    # & 
    # (glued_df["event_name"] == "app_update")
].event_name.unique()


array(['create_parti_home', 'signup_code', 'signup_phone_number',
       'signup_birthday', 'signup_profile', 'create_parti_start',
       'create_parti_setting', 'signup_theme', 'create_parti_done',
       'parti_unmute_talk', 'parti_mute_talk', 'parti_mute_music',
       'parti_unmute_music', 'parti_join', 'parti_leave'], dtype=object)

In [159]:
event_names = [
    "parti_unmute_talk",
    # "parti_join",
    # "parti_leave",
    "parti_mute_talk",
    "parti_unmute_music",
    "parti_mute_music",
]
df = glued_df[(glued_df["event_params.key"] == "label") & (glued_df["event_name"].isin(event_names))].copy()

In [120]:
df.head()

,index,event_timestamp,event_name,operating_system,event_params.key,event_params.value.string_value,event_params.value.int_value,event_params.value.float_value,event_params.value.double_value
609,0,2021-12-20 10:32:22.684013,parti_mute_talk,iOS,label,QWNjb3VudDpmNGE5M2FlODJmNTgxMWVjOTllY2FhNjFjZT...,None,None,None
610,1,2021-12-20 10:36:40.798014,parti_mute_music,iOS,label,QWNjb3VudDpmNGE5M2FlODJmNTgxMWVjOTllY2FhNjFjZT...,None,None,None
611,2,2021-12-20 11:09:19.814002,parti_mute_talk,iOS,label,QWNjb3VudDpmNGE5M2FlODJmNTgxMWVjOTllY2FhNjFjZT...,None,None,None
613,4,2021-12-20 07:30:48.606006,parti_mute_music,iOS,label,QWNjb3VudDowNTk4Zjk3MjM1NDYxMWVjOTNjNGNhZTA2Zm...,None,None,None
615,6,2021-12-20 07:30:18.609005,parti_mute_talk,iOS,label,QWNjb3VudDowNTk4Zjk3MjM1NDYxMWVjOTNjNGNhZTA2Zm...,None,None,None


In [122]:
df.head()

,index,event_timestamp,event_name,operating_system,event_params.key,event_params.value.string_value,event_params.value.int_value,event_params.value.float_value,event_params.value.double_value
609,0,2021-12-20 10:32:22.684013,parti_mute_talk,iOS,label,QWNjb3VudDpmNGE5M2FlODJmNTgxMWVjOTllY2FhNjFjZT...,None,None,None
610,1,2021-12-20 10:36:40.798014,parti_mute_music,iOS,label,QWNjb3VudDpmNGE5M2FlODJmNTgxMWVjOTllY2FhNjFjZT...,None,None,None
611,2,2021-12-20 11:09:19.814002,parti_mute_talk,iOS,label,QWNjb3VudDpmNGE5M2FlODJmNTgxMWVjOTllY2FhNjFjZT...,None,None,None
613,4,2021-12-20 07:30:48.606006,parti_mute_music,iOS,label,QWNjb3VudDowNTk4Zjk3MjM1NDYxMWVjOTNjNGNhZTA2Zm...,None,None,None
615,6,2021-12-20 07:30:18.609005,parti_mute_talk,iOS,label,QWNjb3VudDowNTk4Zjk3MjM1NDYxMWVjOTNjNGNhZTA2Zm...,None,None,None


In [160]:
df["account_id"] = df["event_params.value.string_value"].apply(get_global_id)

In [161]:
df["party_date"] = df["event_timestamp"].dt.date

In [104]:
df.account_id.unique()

array(['f4a93ae8-2f58-11ec-99ec-aa61ce6d5b83',
       '0598f972-3546-11ec-93c4-cae06ffd4ce7',
       '31f6814c-25b5-11ec-aa51-265742dee0f8',
       '3b0264d6-2b32-11ec-b712-ca535be7787b',
       'f24837d6-0f2d-11ec-94df-9a1e772cbdbb',
       '14663cee-2c9f-11ec-a7a6-ea00471996f7',
       'f1a4e2c0-4be6-11ec-b269-9a58024cb2de',
       'c156674e-5675-11ec-a358-d25bc05994ee',
       '3fa0bc02-3def-11ec-8f73-9a4de6baa275',
       '7fa774aa-4be4-11ec-89a8-d22bdb8453e3',
       '5f9112c2-15d7-11ec-bef7-fe46da0c8a72',
       '3ff9b6fc-5be2-11ec-ace0-362b24406086',
       '50b92d1c-6252-11ec-9907-56dcfa0f4d43',
       '01acb826-5bf7-11ec-ace0-362b24406086',
       'ec386c64-30eb-11ec-8de6-b25fc0b029f1',
       'b236317c-620a-11ec-a373-0a1f94409033',
       '78d6d2f4-04b0-11ec-a18f-f6f40593bc15',
       '5beb93ea-4bdd-11ec-acc9-9a58024cb2de',
       '63b6ac52-5c6f-11ec-90b4-7a717ba63af1',
       'dc206810-5d08-11ec-859d-9a7c59f654ca',
       'b250542c-4bd4-11ec-89a8-d22bdb8453e3'], dtype=object

In [162]:
df = pd.merge(df, users[["account_id", "id", "username"]], how="left", left_on="account_id", right_on="account_id")
df.head()

,index,event_timestamp,event_name,operating_system,event_params.key,event_params.value.string_value,event_params.value.int_value,event_params.value.float_value,event_params.value.double_value,account_id,party_date,id,username
0,0,2021-12-22 00:52:39.658000,parti_unmute_talk,iOS,label,QWNjb3VudDpjMTU2Njc0ZTU2NzUxMWVjYTM1OGQyNWJjMD...,None,None,None,c156674e-5675-11ec-a358-d25bc05994ee,2021-12-22,NaN,NaN
1,1,2021-12-22 00:56:20.254008,parti_mute_talk,iOS,label,QWNjb3VudDpjMTU2Njc0ZTU2NzUxMWVjYTM1OGQyNWJjMD...,None,None,None,c156674e-5675-11ec-a358-d25bc05994ee,2021-12-22,NaN,NaN
2,2,2021-12-22 00:56:23.628009,parti_unmute_talk,iOS,label,QWNjb3VudDpjMTU2Njc0ZTU2NzUxMWVjYTM1OGQyNWJjMD...,None,None,None,c156674e-5675-11ec-a358-d25bc05994ee,2021-12-22,NaN,NaN
3,3,2021-12-22 00:58:36.117020,parti_unmute_talk,iOS,label,QWNjb3VudDpjMTU2Njc0ZTU2NzUxMWVjYTM1OGQyNWJjMD...,None,None,None,c156674e-5675-11ec-a358-d25bc05994ee,2021-12-22,NaN,NaN
4,4,2021-12-22 00:59:51.405025,parti_unmute_talk,iOS,label,QWNjb3VudDpjMTU2Njc0ZTU2NzUxMWVjYTM1OGQyNWJjMD...,None,None,None,c156674e-5675-11ec-a358-d25bc05994ee,2021-12-22,NaN,NaN


In [163]:
df = df.dropna(subset=["username", "id"])

In [164]:
activities = pd.pivot_table(df, index=["party_date", "username"], values=["event_timestamp"], columns=["event_name"], aggfunc='count').reset_index()

In [165]:
activities.columns = ["_".join(a) for a in activities.columns.to_flat_index()]

In [166]:
activities

,party_date_,username_,event_timestamp_parti_mute_music,event_timestamp_parti_mute_talk,event_timestamp_parti_unmute_music,event_timestamp_parti_unmute_talk
0,2021-12-16,dft_dev,NaN,2.0,NaN,NaN
1,2021-12-16,freewillyguy9,NaN,8.0,NaN,1.0
2,2021-12-17,bigZoom216,1.0,5.0,NaN,1.0
3,2021-12-17,dft_dev,NaN,5.0,NaN,3.0
4,2021-12-17,freewillyguy9,NaN,8.0,NaN,1.0
5,2021-12-17,marvelfan03,NaN,7.0,NaN,1.0
6,2021-12-17,rio,NaN,5.0,NaN,1.0
7,2021-12-17,sydxeyx,NaN,6.0,1.0,4.0
8,2021-12-22,Doge59,7.0,35.0,2.0,16.0
9,2021-12-22,Iam9777,8.0,30.0,6.0,6.0


In [168]:
activities_df = pd.merge(basic_dff_, activities, how="left", left_on=["start_date_", "username_pu_"], right_on=["party_date_", "username_"])

In [169]:
activities_df.head()

,week_,year_,start_date_,username_pu_,phone_,name,party_id_host,party_id_guest,party_duration_host_text,party_duration_guest_text,all_duration_text,party_duration_host,party_duration_guest,all_duration,duration_range,party_date_,username_,event_timestamp_parti_mute_music,event_timestamp_parti_mute_talk,event_timestamp_parti_unmute_music,event_timestamp_parti_unmute_talk
0,50,2021,2021-12-18,sydxeyx,+13524609925,Sydney Honeycutt,1.0,NaN,0 days 0h0m8s,,0 days 0h0m8s,8.112,0.000,8.112,< 30m,NaN,NaN,NaN,NaN,NaN,NaN
1,50,2021,2021-12-18,Joseph,+13526301846,NaN,NaN,1.0,,,,0.000,0.001,0.001,< 30m,NaN,NaN,NaN,NaN,NaN,NaN
2,50,2021,2021-12-17,zhastings17,+16162381663,NaN,NaN,1.0,,,,0.000,0.000,0.000,< 30m,NaN,NaN,NaN,NaN,NaN,NaN
3,50,2021,2021-12-17,the0nly1con,+19545950848,AJ Drayton,1.0,1.0,0 days 0h2m11s,,0 days 0h2m11s,131.713,0.000,131.713,< 30m,NaN,NaN,NaN,NaN,NaN,NaN
4,50,2021,2021-12-17,sydxeyx,+13524609925,Sydney Honeycutt,1.0,2.0,0 days 0h39m13s,0 days 21h4m38s,0 days 21h43m52s,2353.856,75878.313,78232.169,>1h,2021-12-17,sydxeyx,NaN,6.0,1.0,4.0


In [170]:
set_with_dataframe(
    worksheet,
    activities_df[
        [
            "event_timestamp_parti_mute_music",
            "event_timestamp_parti_unmute_music",
            "event_timestamp_parti_mute_talk",
            "event_timestamp_parti_unmute_talk",
        ]
    ],
    row=2,
    col=22,
    include_column_header=False,
)
